In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d odins0n/ucf-crime-dataset

Dataset URL: https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset
License(s): CC0-1.0
100% 11.0G/11.0G [01:38<00:00, 177MB/s]
100% 11.0G/11.0G [01:38<00:00, 120MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/ucf-crime-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Set parameters
IMG_SIZE = 224  # Image size for resizing
BATCH_SIZE = 32
NUM_CLASSES = 14  # Number of classes in the UCF Crime dataset

In [8]:
data_dir = '/content/Train'

# ImageDataGenerator for loading images
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

# Training data generator
train_generator = datagen.flow_from_directory(data_dir,
                                              target_size=(IMG_SIZE, IMG_SIZE),
                                              batch_size=BATCH_SIZE,
                                              class_mode='categorical',
                                              subset='training')

# Validation data generator
val_generator = datagen.flow_from_directory(data_dir,
                                            target_size=(IMG_SIZE, IMG_SIZE),
                                            batch_size=BATCH_SIZE,
                                            class_mode='categorical',
                                            subset='validation')

Found 1013081 images belonging to 14 classes.
Found 253264 images belonging to 14 classes.


In [10]:
# Load the ResNet50 model without the top classification layers
base_model = tf.keras.applications.ResNet50(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [ ]:
# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# Train the top layers
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=20,
                    callbacks=[early_stopping, reduce_lr])

# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_generator,
                         validation_data=val_generator,
                         epochs=20,
                         callbacks=[early_stopping, reduce_lr])


In [ ]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Save the model
model.save('ucf_crime_image_classifier.h5')
